In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:40237 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


In [77]:
from sklearn.feature_extraction.text import HashingVectorizer,CountVectorizer,TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import dask.array as da
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
dirname='./data'

In [3]:
os.path.join(dirname,'categories_string.csv')

'./data/categories_string.csv'

In [4]:
dd.read_json?

In [52]:
# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv(os.path.join(dirname,'categories_string.csv'))['0'].to_dict()

# Importation du jeu de données d'entrainement
trainX=dd.read_json(os.path.join(dirname,'train.json'),lines=False).set_index('Id',n_partitions=10).categorize(columns=['gender'])

trainTarget=dd.read_csv(os.path.join(dirname, 'train_label.csv')).set_index('Id').categorize(columns=['Category'])

# Concatenation en un seul DataFrame pour visualiser
#trainDF= trainX.assign(Y = trainTarget.Category)

#trainDF['description']=trainDF.apply(lambda x:prepareTxt(x['description']),axis=1,meta=(None, 'object'))

In [55]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX, trainTarget.Category,shuffle=True,test_size=0.3)

In [79]:
#vectorizer = HashingVectorizer(n_features=2**4)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train.description)
X_test = vectorizer.fit_transform(X_test.description)

AttributeError: description not found

In [70]:
classes = da.unique(y_train).compute()


AttributeError: 'Series' object has no attribute 'ravel'

In [71]:
from sklearn.linear_model import SGDClassifier

est = SGDClassifier(loss='log', penalty='l2')

In [74]:
from dask_ml.wrappers import Incremental

inc = Incremental(est,scoring='f1_macro')

In [75]:
inc.fit(X_train, y_train, classes=list(classes.values))

UnboundLocalError: local variable 'nblocks' referenced before assignment

In [ ]:
inc.score(X_test, y_test)

In [ ]:
inc.predict(X_test)  # Predict produces lazy dask arrays

In [ ]:
inc.score(X_test, y_test)